<a href="https://colab.research.google.com/github/peterpanw/Machine_learning/blob/main/torch_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch

In [10]:
torch.__version__

'1.9.0+cu111'

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os

In [12]:
#对训练集进行数据增强 包括左右翻转，灰度随机变换，并变为张量
#测试集就不需要了 只需转换为张量即可
transform=transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
     transforms.RandomGrayscale(),
     transforms.ToTensor()])
transform1=transforms.Compose(
    [
     transforms.ToTensor()])

In [13]:
#下载数据
trainset=torchvision.datasets.FashionMNIST(root='./data',train=True,
                   download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=100,
                   shuffle=True,num_workers=2)
testset=torchvision.datasets.FashionMNIST(root='./data',train=False,
                   download=True,transform=transform1)
testloader=torch.utils.data.DataLoader(testset,batch_size=50,
                   shuffle=False,num_workers=2)

In [14]:
classes = ('T-shirt', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [15]:
#搭建网络
class Net(nn.Module):


    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,64,1,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.fc5 = nn.Linear(128*8*8,512)
        self.drop1 = nn.Dropout2d()
        self.fc6 = nn.Linear(512,10)


    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        #print(" x shape ",x.size())
        x = x.view(-1,128*8*8)
        x = F.relu(self.fc5(x))
        x = self.drop1(x)
        x = self.fc6(x)

        return x

    def train_sgd(self,device,epochs=100):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)

        path = 'weights.tar'
        initepoch = 0

        if os.path.exists(path) is not True:
            loss = nn.CrossEntropyLoss()
            # optimizer = optim.SGD(self.parameters(),lr=0.01)

        else:
            checkpoint = torch.load(path)
            self.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            initepoch = checkpoint['epoch']
            loss = checkpoint['loss']


        for epoch in range(initepoch,epochs):  # loop over the dataset multiple times
            timestart = time.time()

            running_loss = 0.0
            total = 0
            correct = 0
            for i, data in enumerate(trainloader, 0):
                # get the inputs
                inputs, labels = data
                inputs, labels = inputs.to(device),labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                l = loss(outputs, labels)
                l.backward()
                optimizer.step()

                # print statistics
                running_loss += l.item()
                # print("i ",i)
                if i % 500 == 499:  # print every 500 mini-batches
                    print('[%d, %5d] loss: %.4f' %
                          (epoch, i, running_loss / 500))
                    running_loss = 0.0
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    print('Accuracy of the network on the %d tran images: %.3f %%' % (total,
                            100.0 * correct / total))
                    total = 0
                    correct = 0
                    torch.save({'epoch':epoch,
                                'model_state_dict':net.state_dict(),
                                'optimizer_state_dict':optimizer.state_dict(),
                                'loss':loss
                                },path)

            print('epoch %d cost %3f sec' %(epoch,time.time()-timestart))

        print('Finished Training')

    def test(self,device):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: %.3f %%' % (
                100.0 * correct / total))

In [16]:
if __name__ == "__main__":

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = Net()
    net = net.to(device)
    net.train_sgd(device,30)
    net.test(device)

[0,   499] loss: 0.2900
Accuracy of the network on the 100 tran images: 91.000 %
epoch 0 cost 24.886730 sec
[1,   499] loss: 0.2434
Accuracy of the network on the 100 tran images: 87.000 %
epoch 1 cost 24.898417 sec
[2,   499] loss: 0.2190
Accuracy of the network on the 100 tran images: 88.000 %
epoch 2 cost 25.131361 sec
[3,   499] loss: 0.1971
Accuracy of the network on the 100 tran images: 93.000 %
epoch 3 cost 24.918609 sec
[4,   499] loss: 0.1817
Accuracy of the network on the 100 tran images: 91.000 %
epoch 4 cost 24.974836 sec
[5,   499] loss: 0.1676
Accuracy of the network on the 100 tran images: 95.000 %
epoch 5 cost 24.885029 sec
[6,   499] loss: 0.1550
Accuracy of the network on the 100 tran images: 94.000 %
epoch 6 cost 24.899967 sec
[7,   499] loss: 0.1413
Accuracy of the network on the 100 tran images: 96.000 %
epoch 7 cost 24.975025 sec
[8,   499] loss: 0.1342
Accuracy of the network on the 100 tran images: 93.000 %
epoch 8 cost 24.911045 sec
[9,   499] loss: 0.1214
Accu